<a href="https://colab.research.google.com/github/Sejong-Kaggle-Challengers/jeongmin/blob/main/%ED%87%B4%EA%B7%BC%EC%8B%9C%EA%B0%84_%EB%B2%84%EC%8A%A4%EC%8A%B9%EC%B0%A8%EC%9D%B8%EC%9B%90_%EC%98%88%EC%B8%A1_ver1_%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- id : 해당 데이터에서의 고유한 ID(train, test와의 중복은 없음)
- date : 날짜
- bus_route_id : 노선ID
- in_out : 시내버스, 시외버스 구분
- station_code : 해당 승하차 정류소의 ID
- station_name : 해당 승하차 정류소의 이름
- latitude : 해당 버스 정류장의 위도 (같은 정류장 이름이어도 버스의 진행 방향에 따라 다를 수 있음)
- longitude : 해당 버스 정류장의 경도 (같은 정류장 이름이어도 버스의 진행 방향에 따라 다를 수 있음)
- X~Y_ride : X:00:00부터 X:59:59까지 승차한 인원 수
- X~Y_takeoff : X:00:00부터 X:59:59까지 하차한 인원 수
- 18~20_ride : 18:00:00부터 19:59:59까지 승차한 인원 수 (train data에만 존재)

bus_route_id / date / station_code 겹치는 것을 사용

bus_bts 에서 geton_date - getoff_date(Nan값 많음)
대부분 동일 -> geton_date만 사용? 

하차 부분이 안찍힌 경우가 다 Nan값으로 떠있기 때문에
채우는 것이 더 안좋을지 뺴는 부분이 좋을지 ? ? 
-> (895736)Nan값을 일단 제거하고 사용

-> 탑승한 시간별로 그 승객들의 탑승하고 있는 시간의 평균..? 이용?
user_category 사용

station_name / latitude와longitude
합쳐서 사용
지명 - 1/지명 - 2 로 변경하거나
위도 경도만 사용 ? 등등 ..

in_out -> 0과 1로 바꾸기

date -> 공휴일(명절) 유무 / 요일

모델은 LR -> NN ?

#데이터 업로드

In [2]:
!unzip data13.zip

Archive:  data13.zip
  inflating: 행정_법정동 중심좌표.xlsx  
  inflating: test.csv                
  inflating: train.csv               
  inflating: submission_sample.csv   
  inflating: bus_bts.csv             


In [67]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test.head()

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff
0,415423,2019-10-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,4.0,4.0,7.0,2.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,415424,2019-10-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,6.0,6.0,1.0,8.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0
2,415425,2019-10-01,4270000,시외,432,정존마을,33.48181,126.47352,2.0,4.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,415426,2019-10-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,1.0,11.0,18.0,8.0,26.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0
4,415427,2019-10-01,4270000,시내,1636,롯데호텔,33.24872,126.41032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [63]:
train.shape

(415423, 23)

###bus_bts(아직 고려 X)

In [ ]:
bus_bts = pd.read_csv("bus_bts.csv")
bus_bts.head()

,user_card_id,bus_route_id,vhc_id,geton_date,geton_time,geton_station_code,geton_station_name,getoff_date,getoff_time,getoff_station_code,getoff_station_name,user_category,user_count
0,1.010010e+15,23000000,149793674,2019-09-10,06:34:45,360,노형오거리,2019-09-10,07:10:31,592.0,화북초등학교,1,1
1,1.010010e+15,23000000,149793674,2019-09-10,06:34:58,360,노형오거리,2019-09-10,06:56:27,3273.0,고산동산(광양방면),1,1
2,1.019160e+15,21420000,149793535,2019-09-10,07:19:07,2495,동광환승정류장4(제주방면),2019-09-10,07:40:29,431.0,정존마을,4,1
3,1.019150e+15,21420000,149793512,2019-09-09,09:14:47,3282,대정환승정류장(대정읍사무소),2019-09-09,10:02:46,431.0,정존마을,1,1
4,1.010010e+15,21420000,149793512,2019-09-09,09:28:53,2820,삼정지에듀,2019-09-09,10:21:37,2972.0,제주국제공항(종점),4,1


In [ ]:
bus_bts.shape

(2409414, 13)

In [ ]:
bus_bts.isnull().sum()

user_card_id                0
bus_route_id                0
vhc_id                      0
geton_date                  0
geton_time                  0
geton_station_code          0
geton_station_name         49
getoff_date            895736
getoff_time            895736
getoff_station_code    895736
getoff_station_name    895775
user_category               0
user_count                  0
dtype: int64

In [ ]:
train_unique = []
for i in train.columns.values:
  train_unique.append(len(train[i].unique()))
unique_train = pd.DataFrame()
unique_train['Columns'] = list(train.columns)
unique_train['Unique_value'] = train_unique

In [ ]:
unique_train

,Columns,Unique_value
0,id,415423
1,date,30
2,bus_route_id,613
3,in_out,2
4,station_code,3563
5,station_name,1961
6,latitude,3278
7,longitude,3446
8,6~7_ride,47
9,7~8_ride,80


#데이터 전처리
1. 날짜
- 명절 유무 column 생성하기 (2019-09-12 ~ 2019-09-14 : 추석)
- 요일 column 생성하기
2. 	in_out(시내/시외)
- 범주형을 정수형으로 변경
3. bus_route_id
- 노선 ID 값이 8자리로 매우 큰 수 -> 10000으로 나누어 줌
4. station_name 행 제거
- station_code를 정렬해본 결과 같은 station_code끼리 이름과 위도 경도가 모두 일치

id와 date는 제외하고 학습에 사용
이후 데이터 정규화 StandardScaler 사용

In [4]:
#날짜 정보 알아보기
train['date'].unique()

array(['2019-09-01', '2019-09-02', '2019-09-03', '2019-09-04',
       '2019-09-05', '2019-09-06', '2019-09-07', '2019-09-08',
       '2019-09-09', '2019-09-10', '2019-09-11', '2019-09-12',
       '2019-09-13', '2019-09-14', '2019-09-15', '2019-09-16',
       '2019-09-17', '2019-09-18', '2019-09-19', '2019-09-20',
       '2019-09-21', '2019-09-22', '2019-09-23', '2019-09-24',
       '2019-09-25', '2019-09-26', '2019-09-27', '2019-09-28',
       '2019-09-29', '2019-09-30'], dtype=object)

In [68]:
test['date'].unique()

array(['2019-10-01', '2019-10-02', '2019-10-03', '2019-10-04',
       '2019-10-05', '2019-10-06', '2019-10-07', '2019-10-08',
       '2019-10-09', '2019-10-10', '2019-10-11', '2019-10-12',
       '2019-10-13', '2019-10-14', '2019-10-15', '2019-10-16'],
      dtype=object)

In [69]:
#요일 
import datetime
train['dayofweek'] = pd.to_datetime(train['date']).dt.weekday
test['dayofweek'] = pd.to_datetime(test['date']).dt.weekday
train['dayofweek']
#0 -> 월 ~ 6 -> 일

0         6
1         6
2         6
3         6
4         6
         ..
415418    0
415419    0
415420    0
415421    0
415422    0
Name: dayofweek, Length: 415423, dtype: int64

In [70]:
#공휴일 -- 공휴일이면 1 아니면 0
holi = pd.Series(['2019-09-12','2019-09-13', '2019-09-14'])
train['holiday'] = train['date'].apply(lambda x : 1 if holi[0] == x or holi[1] == x or holi[2] == x else 0 )
holi = pd.Series(['2019-10-03','2019-10-09'])
test['holiday'] = test['date'].apply(lambda x : 1 if holi[0] == x or holi[1] == x else 0 )
print(train['holiday'].value_counts())
print(test['holiday'].value_counts())

0    384704
1     30719
Name: holiday, dtype: int64
0    203564
1     24606
Name: holiday, dtype: int64


In [73]:
#in_out 숫자로 변경 (시외 : 1, 시내 : 0)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['in_out'] = le.fit_transform(train['in_out'])
test['in_out'] = le.transform(test['in_out'])
train['in_out']

0         1
1         1
2         1
3         0
4         0
         ..
415418    0
415419    0
415420    0
415421    0
415422    0
Name: in_out, Length: 415423, dtype: int64

In [74]:
train['bus_route_id'] = train['bus_route_id']/10000
test['bus_route_id'] = test['bus_route_id']/10000
train['bus_route_id']

0          427.0
1          427.0
2          427.0
3          427.0
4          427.0
           ...  
415418    3282.0
415419    3282.0
415420    3282.0
415421    3282.0
415422    3282.0
Name: bus_route_id, Length: 415423, dtype: float64

In [75]:
del train['station_name']
del test['station_name']

In [65]:
train.head()

,id,date,bus_route_id,in_out,station_code,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride,dayofweek,holiday
0,0,2019-09-01,427.0,1,344,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,0
1,1,2019-09-01,427.0,1,357,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,6,0
2,2,2019-09-01,427.0,1,432,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6,0
3,3,2019-09-01,427.0,0,1579,33.50577,126.49252,0.0,17.0,6.0,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,6,0
4,4,2019-09-01,427.0,0,1646,33.25579,126.41260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6,0


In [76]:
test.head()

,id,date,bus_route_id,in_out,station_code,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,dayofweek,holiday
0,415423,2019-10-01,427.0,1,344,33.48990,126.49373,4.0,4.0,7.0,2.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0
1,415424,2019-10-01,427.0,1,357,33.48944,126.48508,1.0,6.0,6.0,1.0,8.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,415425,2019-10-01,427.0,1,432,33.48181,126.47352,2.0,4.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,415426,2019-10-01,427.0,0,1579,33.50577,126.49252,1.0,11.0,18.0,8.0,26.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4,415427,2019-10-01,427.0,0,1636,33.24872,126.41032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0


In [117]:
train_x = train.loc[:, "bus_route_id":"holiday"]
del train_x['18~20_ride']
train_y = train['18~20_ride']

test_x = test.loc[:, "bus_route_id":"holiday"]

In [118]:
#데이터 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

#모델 학습

In [119]:
import torch
import numpy as np
import torch.optim as optim

torch.manual_seed(1)

train_x = np.array(train_x)
train_y = np.array(train_y)

train_x = torch.FloatTensor(train_x)
train_y = torch.FloatTensor(train_y).unsqueeze(1)

train_y

tensor([[0.],
        [5.],
        [2.],
        ...,
        [0.],
        [0.],
        [0.]])

In [120]:
#test_x = test.loc[:, "bus_route_id":"holiday"]
test_x = torch.FloatTensor(np.array(test_x))

In [121]:
train_x.shape

torch.Size([415423, 19])

In [128]:
W = torch.zeros((19, 1), requires_grad= True)
b = torch.zeros(1, requires_grad= True)

optimizer = optim.SGD([W, b], lr = 1e-2)

nb_epochs = 1000

for epoch in range(nb_epochs + 1) :

  hypothesis = train_x.matmul(W) + b
  cost = torch.mean((hypothesis - train_y)**2)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print(epoch, cost.item())

0 23.842741012573242
100 12.484683990478516
200 12.410828590393066
300 12.404206275939941
400 12.403141975402832
500 12.402926445007324
600 12.40287971496582
700 12.402867317199707
800 12.402865409851074
900 12.402864456176758
1000 12.402864456176758


In [131]:
predict = (test_x.matmul(W)+b).detach()

submit = pd.read_csv("submission_sample.csv")

for i in range(len(predict)):
  submit["18~20_ride"][i] = predict[i]
  
submit["18~20_ride"] = submit["18~20_ride"].astype(int)

submit.to_csv("submission.csv", index = False, header = True)

#성능 (오차 정도) : 3.23622